<a href="https://colab.research.google.com/github/muthuraman2002/Blood/blob/main/chatbot_using_chormaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# @title Default title text
!pip install langchain chromadb tiktoken langchain-community transformers tensorflow sentence-transformers



In [4]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
# from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer


In [2]:
# print(embeddings,'try')
# Load your documents (replace with your actual document loading)
file_path = "my_document.txt"  # Replace with your file path
text=""
try:
    with open(file_path, "r") as f:
      # print(f.read(),'reading')
      text = f.read()
except FileNotFoundError:
    with open(file_path, "w") as f:
      f.write("")  # Optionally initialize with an empty string
      text = ""

# texts = [
#     "The quick brown fox jumps over the lazy dog.",
#     "A journey of a thousand miles begins with a single step.",
#     "To be or not to be, that is the question.",
#     "All that glitters is not gold."
# ]
# print(pipeline)
# Split the text into chunks

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)
# Create embeddings and store them in Chroma
# model= SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # Use HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# embeddings = model.encode(texts)
# print(texts,'texts')
metadatas = [{"source": str(i)} for i in range(len(texts))]
# print(texts,Chroma.from_texts())
# try:
# docsearch = Chroma.afrom_texts(texts, embeddings, metadatas=metadatas)
docsearch = Chroma.from_texts(
    texts=texts,
    embedding=embeddings,
    metadatas=metadatas,
    persist_directory="./chroma_data"  # Optional for persistence
)
# except Exception as e:
#     print(f"An error occurred: {e}",'exception')
model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100,pad_token_id = 50256 )
llm = HuggingFacePipeline(pipeline=pipe)

<ipython-input-2-6beaa2c0752f>:27: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # Use HuggingFaceEmbeddings
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ValueError: Expected Embedings to be non-empty list or numpy array, got [] in upsert.

In [1]:

# Initialize the language model using Hugging Face's pipeline
# hf_pipeline = pipeline("text-generation", model="gpt2")
# print(HuggingFacePipeline,'tokeniser')
# llm = HuggingFacePipeline(pipeline=hf_pipeline)
# pad_token_id=hf_pipeline.tokenizer.eos_token_id
# print(pad_token_id)
# Create the conversational retrieval chain
# try:
qa = ConversationalRetrievalChain.from_llm(llm, docsearch.as_retriever())
# except Exception as e:
    # print(f'error occured :{e}','qa')
    # Start the chat loop
# print(qa)
try:
    chat_history = []
    while True:
        query = input("Ask a question (or type 'exit' to quit): ")
        if query.lower() == "exit":
            break

        result = qa({"question": query, "chat_history": chat_history})
        print(result["answer"])
        chat_history.append((query, result["answer"]))
    print(f"Chat History: {chat_history}")
except Exception as e:
    print(e)
    # print('hii')


NameError: name 'ConversationalRetrievalChain' is not defined